In [ ]:
!pip install llama_index
!pip install langchain

In [6]:
import os

os.environ['OPENAI_API_KEY'] = "API_KEY"

In [ ]:

!pip install pypdf


In [9]:
from llama_index import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["/content/sample_data/calcrim_2023_edition.pdf"])
documents = reader.load_data()

In [ ]:
from langchain.llms import OpenAI
!pip install llama-index

from llama_index import LLMPredictor, GPTVectorStoreIndex, PromptHelper


# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.1, model_name="text-davinci-002"))

max_input_size = 4896
num_outputs = 512
max_chunk_overlap = 20
chunk_size_limit = 681
prompt_helper = PromptHelper(max_input_size, num_outputs, chunk_overlap_ratio= 0.1, chunk_size_limit=chunk_size_limit)

custom_LLM_index =GPTVectorStoreIndex(
    documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper
)

In [32]:
import openai
import json

class Chatbot:
    def __init__(self, api_key, index):
        self.index = index
        openai.api_key = api_key

    def generate_response_from_text(self, user_input):
        prompt = f"User: {user_input} tell only one best matching penal code for the crime in this report?"
        query_engine = self.index.as_query_engine()  # Use self.index instead of index
        response = query_engine.query(prompt)  # Use prompt instead of user_input

        message = {"content": response.response}
        return message


In [33]:
# Swap out your index below for whatever knowledge base you want
bot = Chatbot("API_KEY", index=custom_LLM_index)

!pip install python-docx
from docx import Document
from google.colab import files
import io

def upload_file():
    uploaded_files = files.upload()

    if uploaded_files:
        file_content = list(uploaded_files.values())[0]
        return file_content
    else:
        return None

while True:
    upload_option = input("Do you want to upload a file? (yes/no): ").lower()

    if upload_option == "no":
        break

    if upload_option == "yes":
        file_content = upload_file()
        if file_content:
            try:
                # Use python-docx to read the document from bytes
                doc = Document(io.BytesIO(file_content))

                # Extract text from DOCX
                paragraphs = [paragraph.text for paragraph in doc.paragraphs]
                user_input = "\n".join(paragraphs)

                # Generate a response using a 'bot' object
                response = bot.generate_response_from_text(user_input)
                print(f"Bot: {response['content']}")

            except Exception as e:
                print(f"Error: Unable to read the Word document content. {str(e)}")
        else:
            print("File upload canceled or failed. Please try again.")
    else:
        print("Invalid option. Please enter 'yes' or 'no'.")


Do you want to upload a file? (yes/no): yes


Saving a.docx to a (9).docx
Bot: Penal Code section 186.22(a) - Active Participation in Criminal Street Gang
Do you want to upload a file? (yes/no): no
